# CAO points analysis in 2019 - 2021  


The Central Applications Office processes applications for undergraduate courses in Irish Higher Education Institutions (HEIs).

In [1]:
# Convenient HTTP requests.
import requests as rq

# Regular expressions.
import re

# Dates and times.
import datetime as dt

# Data frames.
import pandas as pd

# For downloading.
import urllib.request as urlrq

# For loading PDFs into pandas
import camelot

In [2]:
# Get the current date and time.
now = dt.datetime.now()

# Format as a string.
nowstr = now.strftime('%Y%m%d_%H%M%S')

<br>

## 2021 Points Level 8

Taken from [cao.ie](http://www2.cao.ie/points/l8.php)
***

In [3]:
# Fetch the CAO points URL.
resp = rq.get('http://www2.cao.ie/points/l8.php')

# Testing of we getting expected response (200 means OK).
resp

<Response [200]>

<br>

## Save original data set
***

In [4]:
# Create a file path for the original data.
pathhtml = 'data/cao2021_' + nowstr + '.html'

<br>

**Fixing encoding error**

If we inspect the page in hte browser, server says we should decode as per:  

```
Content-Type: text/html; charset=iso-8859-1
```
     
However, one line uses #x96 character which isn't defined in iso-8859-1.
Therefore to fix this issue we going to use very similar standart cp1252,
which includes #x96 character

In [5]:
# The server uses the wrong encoding, fix it.
original_encoding = resp.encoding

# Change to cp1252.
resp.encoding = 'cp1252'

In [6]:
# Save the original html file.
with open(pathhtml, 'w') as f:
    f.write(resp.text)

<br>

## Use regular expressions to select lines we want
***

In [7]:
# Compile the regular expression for matching lines.
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

<br>

### Loop through the lines of the response
***

In [8]:
# The file path for the csv file.
path2021 = 'data/cao2021_csv_' + nowstr + '.csv'

In [9]:
# Keep track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path2021, 'w') as f:
    # Write a header row.
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    # Loop through lines of the response.
    for line in resp.iter_lines():
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57].strip()
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print (f'Total number of lines is {no_lines}.')

Total number of lines is 949.


<br>

**NB:** it was verified as of 03/11/2021 that there were 949 courses exactly in the CAO 2021 points list.
***

In [10]:
df2021 = pd.read_csv(path2021, encoding='cp1252')

In [11]:
df2021.iloc[162]

code                                                 CK701
title       Medicine (Undergraduate Entry - HPAT required)
pointsR1                                              #738
pointsR2                                             #737*
Name: 162, dtype: object

In [12]:
# Removing # and * from pointsR1 and pointsR2 fileds 
cols = ['pointsR1', 'pointsR2']
for col in cols:
    df2021[col] = df2021[col].map(lambda x: str(x).lstrip('#').rstrip('*'))
df2021.iloc[162]

code                                                 CK701
title       Medicine (Undergraduate Entry - HPAT required)
pointsR1                                               738
pointsR2                                               737
Name: 162, dtype: object

<br>

## 2020 Points all levels

https://www.cao.ie/index.php?page=points&p=2020
***

In [13]:
# URL to CAO points for 2020
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

<br>

### Save Original File
***

In [14]:
# Create a file path for the original data.
pathxlsx = 'data/cao2020_' + nowstr + '.xlsx'

# Save retrieved URL to the file
urlrq.urlretrieve(url2020, pathxlsx)

('data/cao2020_20211227_143139.xlsx',
 <http.client.HTTPMessage at 0x7ffe468f21c0>)

<br>

### Load Spreadsheet using pandas
***

In [15]:
# Download and parse the excel spreadsheet. After checking source data in excel, only selecting columns we might need.
df2020 = pd.read_excel(url2020, skiprows=10, usecols=[1,2,3,5,10,11])

In [16]:
df2020

,COURSE TITLE,COURSE CODE2,R1 POINTS,R2 POINTS,LEVEL,HEI
0,International Business,AC120,209,NaN,8,American College
1,Liberal Arts,AC137,252,NaN,8,American College
2,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,8,National College of Art and Design
3,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,8,National College of Art and Design
4,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,8,National College of Art and Design
...,...,...,...,...,...,...
1459,Manufacturing Engineering,WD208,188,NaN,7,Waterford Institute of Technology
1460,Software Systems Development,WD210,279,NaN,8,Waterford Institute of Technology
1461,Creative Computing,WD211,271,NaN,8,Waterford Institute of Technology
1462,Recreation and Sport Management,WD212,270,NaN,8,Waterford Institute of Technology


In [17]:
# Spot check a random row.
df2020.iloc[753]

COURSE TITLE    Road Transport Technology and Management
COURSE CODE2                                       LC286
R1 POINTS                                            264
R2 POINTS                                            NaN
LEVEL                                                  7
HEI                     Limerick Institute of Technology
Name: 753, dtype: object

In [18]:
# Spot check the last row.
df2020.iloc[-1]

COURSE TITLE    Mechanical and Manufacturing Engineering
COURSE CODE2                                       WD230
R1 POINTS                                            253
R2 POINTS                                            NaN
LEVEL                                                  8
HEI                    Waterford Institute of Technology
Name: 1463, dtype: object

In [19]:
# Select only level 8 courses
df2020_l8 = df2020.loc[df2020['LEVEL'] == 8]
df2020_l8

,COURSE TITLE,COURSE CODE2,R1 POINTS,R2 POINTS,LEVEL,HEI
0,International Business,AC120,209,NaN,8,American College
1,Liberal Arts,AC137,252,NaN,8,American College
2,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,8,National College of Art and Design
3,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,8,National College of Art and Design
4,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,8,National College of Art and Design
...,...,...,...,...,...,...
1455,Arts (options),WD200,AQA,AQA,8,Waterford Institute of Technology
1460,Software Systems Development,WD210,279,NaN,8,Waterford Institute of Technology
1461,Creative Computing,WD211,271,NaN,8,Waterford Institute of Technology
1462,Recreation and Sport Management,WD212,270,NaN,8,Waterford Institute of Technology


In [20]:
# Dropping last two columns as we don't need them after we have already selected only level 8 courses
df2020_l8 = df2020_l8.drop(columns=['LEVEL', 'HEI'])
# Resetting indexes of the rows so they are unique and ordered 
df2020_l8 = df2020_l8.reset_index(drop=True)

In [21]:
df2020_l8 = df2020_l8.rename(columns={"COURSE TITLE": "title", "COURSE CODE2": "code", "R1 POINTS": "pointsR1", "R2 POINTS": "pointsR2"})
df2020_l8

,title,code,pointsR1,pointsR2
0,International Business,AC120,209,NaN
1,Liberal Arts,AC137,252,NaN
2,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN
3,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN
4,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN
...,...,...,...,...
1022,Arts (options),WD200,AQA,AQA
1023,Software Systems Development,WD210,279,NaN
1024,Creative Computing,WD211,271,NaN
1025,Recreation and Sport Management,WD212,270,NaN


In [22]:
# Removing # and * from pointsR1 and pointsR2 fileds 
cols = ['pointsR1', 'pointsR2']
for col in cols:
    df2020_l8[col] = df2020_l8[col].map(lambda x: str(x).lstrip('#+matric ').rstrip('*'))

In [23]:
# Create a file path for the pandas data.
path2020 = 'data/cao2020_' + nowstr + '.csv'
path2020_l8 = 'data/cao2020_l8_' + nowstr + '.csv'

# Save pandas data frame to disk.
df2020.to_csv(path2020)
df2020_l8.to_csv(path2020_l8)

<br>

## 2019 Points

https://www.cao.ie/index.php?page=points&p=2019
***

In [24]:
# Reading PDF that was dowloaded from CAO with camelot, as by default camelot only looks at the first page of the file
# here we specify that we want all pages
cao_tables = camelot.read_pdf('data/cao2019.pdf', pages='1-end')

In [25]:
# Testing that data is now loaded ( we can see there are 18 tables/pages)
cao_tables

<TableList n=18>

In [26]:
# Checking the accuracy on the parsing (this can be checked for each page) 
cao_tables[2].parsing_report

{'accuracy': 100.0, 'whitespace': 2.27, 'order': 1, 'page': 3}

<br>

We can export out tables to csv file now, but camelot will put each table in a diffeent file whish isn't convinient for us, so we need to have a walkaround to combine everything into one file
 ```
cao_tables.export('data/cao2019_' + nowstr + '.csv', f='csv')
```

In [27]:
# Here we using for loop to go through tables, convert them to dataframes and append everything 
# Idea for using appending method was taken from [1]
df2019 = pd.DataFrame()
i=0
for table in cao_tables:
    df = cao_tables[i].df
    i=i + 1
    df2019 = df2019.append(df)
df2019

,0,1,2,3
0,Course Code INSTITUTION and COURSE,,EOS,Mid
1,,Athlone Institute of Technology,,
2,AL801,Software Design with Virtual Reality and Gaming,304,328
3,AL802,Software Design with Cloud Computing,301,306
4,AL803,Software Design with Mobile Apps and Connected...,309,337
...,...,...,...,...
37,WD200,Arts (options),221,296
38,WD210,Software Systems Development,271,329
39,WD211,Creative Computing,275,322
40,WD212,Recreation and Sport Management,274,311


In [28]:
# Renaming columns
df2019 = df2019.rename(columns={0: "code", 1: "title", 2: "pointsR1", 3: "pointsR2"})

In [29]:
# Deleting first two rows
df2019 = df2019.drop([0,1])

In [30]:
df2019

,code,title,pointsR1,pointsR2
2,AL801,Software Design with Virtual Reality and Gaming,304,328
3,AL802,Software Design with Cloud Computing,301,306
4,AL803,Software Design with Mobile Apps and Connected...,309,337
5,AL805,Network Management and Cloud Infrastructure,329,442
6,AL810,Quantity Surveying,307,349
...,...,...,...,...
37,WD200,Arts (options),221,296
38,WD210,Software Systems Development,271,329
39,WD211,Creative Computing,275,322
40,WD212,Recreation and Sport Management,274,311


In [31]:
# Resetting indexes of the rows so they are unique and ordered 
df2019 = df2019.reset_index(drop=True)

We can find rows where column code is an empty string, those rows are the names of the college which we do not need, so we going to drop those rows.

In [32]:
df2019 = df2019.drop(df2019[df2019['code'] == ''].index)

In [33]:
df2019

,code,title,pointsR1,pointsR2
0,AL801,Software Design with Virtual Reality and Gaming,304,328
1,AL802,Software Design with Cloud Computing,301,306
2,AL803,Software Design with Mobile Apps and Connected...,309,337
3,AL805,Network Management and Cloud Infrastructure,329,442
4,AL810,Quantity Surveying,307,349
...,...,...,...,...
925,WD200,Arts (options),221,296
926,WD210,Software Systems Development,271,329
927,WD211,Creative Computing,275,322
928,WD212,Recreation and Sport Management,274,311


In [34]:
# Removing # and * from pointsR1 and pointsR2 fileds 
cols = ['pointsR1', 'pointsR2']
for col in cols:
    df2019[col] = df2019[col].map(lambda x: str(x).lstrip('#+matric ').rstrip('*'))

In [35]:
# Create a file path for the data.
path2019 = 'data/cao2019_' + nowstr + '.csv'

# Save pandas data frame to disk.
df2019.to_csv(path2019)

<br>

## concat and join functions
***

In [36]:
# Selecting columns code and title from data frame  for 2021 and assigning it to courses2021
courses2021 = df2021[['code', 'title']]
courses2021

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
944,WD211,Creative Computing
945,WD212,Recreation and Sport Management
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [37]:
# Selecting columns COURSE CODE2 and COURSE TITLE from data frame  for 2020 and assigning it to courses2020
courses2020 = df2020[['COURSE CODE2','COURSE TITLE']]
#Changing names of the columns to code and title
courses2020.columns = ['code', 'title']
courses2020

,code,title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1459,WD208,Manufacturing Engineering
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [38]:
# Combining two dataframs into one
allcourses = pd.concat([courses2021, courses2020], ignore_index=True)
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2408,WD208,Manufacturing Engineering
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [39]:
# Now, if we sort our data by 'code' we will see that there are a lof of duplicates, as most of the courses which run in 2020
# were also awailable in 2021
allcourses.sort_values('code')

,code,title
175,AC120,International Business
949,AC120,International Business
950,AC137,Liberal Arts
176,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
...,...,...
2411,WD212,Recreation and Sport Management
2412,WD230,Mechanical and Manufacturing Engineering
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [40]:
# To confirm the titles of the courses were the same, which means those entries are duplicates
allcourses.loc[176]['title']

'Liberal Arts'

In [41]:
allcourses.loc[950]['title']

'Liberal Arts'

In [42]:
# Finds all extra copies of duplicated rows. There are currenntly 738 duplicates in our df
allcourses[allcourses.duplicated()]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
952,AD102,Graphic Design and Moving Image Design (portfo...
955,AD204,Fine Art (portfolio)
956,AD211,Fashion Design (portfolio)
...,...,...
2404,WD200,Arts (options)
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [43]:
# Returns a copy of the data frame with duplciates removed. There are now only 1675 rows remaining. (738+1675 = 2413 rows 
# were in total before we removed duplicates)
allcourses.drop_duplicates()

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2403,WD195,Architectural and Building Information Modelli...
2405,WD205,Molecular Biology with Biopharmaceutical Science
2406,WD206,Electronic Engineering
2407,WD207,Mechanical Engineering


In [44]:
# Finds all extra copies of duplicated rows. Now we are looking for duplicated in column 'code'
allcourses[allcourses.duplicated(subset=['code'])]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
952,AD102,Graphic Design and Moving Image Design (portfo...
953,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
2404,WD200,Arts (options)
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [45]:
# Returns a copy of the data frame with duplciates removed - based only on code. Index will be labeled strating form 0.., n-1
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
1512,WD188,Applied Health Care
1513,WD205,Molecular Biology with Biopharmaceutical Science
1514,WD206,Electronic Engineering
1515,WD207,Mechanical Engineering


#### Web resources used
[1] [www.analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2020/08/how-to-extract-tabular-data-from-pdf-document-using-camelot-in-python/)